In [ ]:
----------working with multiple images--------

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
from PIL import Image
import requests
from io import BytesIO

In [ ]:
# Download and prepare dataset from Kaggle
!pip install kaggle
!kaggle datasets download -d puneet6060/intel-image-classification
!unzip intel-image-classification.zip -d dataset

# Data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification
License(s): copyright-authors
intel-image-classification.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  intel-image-classification.zip
replace dataset/seg_pred/seg_pred/10004.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset/seg_pred/seg_pred/10004.jpg  
replace dataset/seg_pred/seg_pred/10005.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset/seg_pred/seg_pred/10005.jpg  
replace dataset/seg_pred/seg_pred/10012.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset/seg_pred/seg_pred/10012.jpg  
replace dataset/seg_pred/seg_pred/10013.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset/seg_pred/seg_pred/10013.jpg  
replace dataset/seg_pred/seg_pred/10017.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset/seg_pred/seg_pred/10017.jpg  
replace dataset/seg_pred/seg_pred/10021.jpg? [y]es, 

In [ ]:

# Prepare dataset loaders
train_dataset = datasets.ImageFolder(root='dataset/seg_train/seg_train', transform=transform)
test_dataset = datasets.ImageFolder(root='dataset/seg_test/seg_test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:


# Load pretrained MobileNetV2 model
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, len(train_dataset.classes))  # Modify last layer


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

Epoch 1, Loss: 0.2138968774399684
Epoch 2, Loss: 0.20117143881008673


In [ ]:
# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Accuracy: 91.33333333333333%


In [ ]:
# Classify multiple images
sample_images = [ '/content/dataset/seg_test/seg_test/forest/20056.jpg', '/content/dataset/seg_train/seg_train/mountain/10000.jpg']
for img_path in sample_images:
    img = Image.open(img_path)
    img = transform(img).unsqueeze(0).to(device)
    model.eval()
    output = model(img)
    _, pred = torch.max(output, 1)
    print(f"Image: {img_path} | Predicted: {train_dataset.classes[pred.item()]}")


Image: /content/dataset/seg_test/seg_test/forest/20056.jpg | Predicted: forest
Image: /content/dataset/seg_train/seg_train/mountain/10000.jpg | Predicted: mountain
